In [1]:
import sys
sys.path.append('/home/g/Desktop/git/nodules/src/')
sys.path.append('/home/g/Desktop/git/nodules/tf/')
sys.path.append('/home/g/Desktop/git/nodules/keras/')
import init
import kerasUtil as ku
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [78]:
def get3Labels(labels,test=1):
  classLabels = np.zeros((len(labels)/test,3))
  for i in range(len(labels)/test):
    l = labels[i]
    print l
    if l == 'LL' or l == 'LLL' or l == 'LLL':
      classLabels[i,0] = 1
    elif l == 'SS' or l == 'SSS' or l == 'SSSS':
      classLabels[i,1] = 1
    elif l == 'NN' or l == 'NNN' or l == 'NNNN':
      classLabels[i,2] = 1
  return classLabels

def cleanData(images,labels):
  temp = []
  for i in range(len(images)):
    temp.append((images[i],labels[i]))
  images, labels = [], []
  for i in range(len(temp)):
    label = temp[i][1]
    l = label['labels']
    print l
    if l == 'LL' or l == 'LLL' or l == 'LLLL':
      images.append(temp[i][0])
      labels.append(l)
    elif l == 'SS' or l == 'SSS' or l == 'SSSS':
      images.append(temp[i][0])
      labels.append(l)
    elif l == 'NN' or l == 'NNN' or l == 'NNNN':
      images.append(temp[i][0])
      labels.append(l)
  return images, labels

In [73]:
cfg = ['/scr/nodules/no3/test/','/scr/nodules/no3/test/']
testImages, testLabels = init.loadData(cfg)
testImages, testLabels = ku.cleanData(testImages, testLabels)
testImages = ku.to3d(testImages)
testLabels = ku.get3Labels(testLabels)
testImages = ku.convertArray(testImages)

LOADING DATA...


In [ ]:
def getModel():
  model = Sequential()
  model.add(Convolution3D(128,kernel_size=5,strides=(1,2,2),data_format='channels_first',input_shape=(1,16,64,64),kernel_initializer='glorot_normal',use_bias=True))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=(2,2,2)))
  model.add(Dropout(0.5))
  model.add(Convolution3D(192,kernel_size=4,strides=1,data_format='channels_first',kernel_initializer='glorot_normal',use_bias=True))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=(2,2,2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(136))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dropout(0.5))
  model.add(Dense(48))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dropout(0.5))
  model.add(Dense(3))
  model.add(Activation('softmax'))
  return model

In [30]:
model = load_model('../../../ensembleTest1a.h5')
print model.evaluate(testImages,testLabels,batch_size=32)
predict1 = model.predict_proba(testImages,batch_size=32)
predict1c = model.predict_classes(testImages,batch_size=32)

1984/2009 [============================>.] - ETA: 0s 

In [31]:
model = load_model('../../../ensembleTest2a.h5')
print model.evaluate(testImages,testLabels,batch_size=32)
predict2 = model.predict_proba(testImages,batch_size=32)
predict2c = model.predict_classes(testImages,batch_size=32)

1984/2009 [============================>.] - ETA: 0s 

In [33]:
model = load_model('../../../ensembleTest3a.h5')
print model.evaluate(testImages,testLabels,batch_size=32)
predict3 = model.predict_proba(testImages,batch_size=32)
predict3c = model.predict_classes(testImages,batch_size=32)

1984/2009 [============================>.] - ETA: 0s 

In [50]:
print predict1c
print predict2c
print predict3c

[2 1 2 ..., 1 1 2]
[2 1 2 ..., 2 1 2]
[2 1 2 ..., 1 1 2]


In [51]:
predict = np.round((predict1c+predict2c+predict3c)/3.)
print predict

[ 2.  1.  2. ...,  1.  1.  2.]


In [68]:
classes = np.zeros(testLabels.shape)
for i in range(predict.shape[0]):
  classes[i,predict1c[i]] = 1

In [69]:
print classes

[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 ..., 
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [74]:
print testLabels

[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]]


In [75]:
print np.sum(testLabels)
print testLabels.shape[0]

1937.0
2009


In [71]:
predict = np.zeros(classes.shape)
predict[classes[:,:]==testLabels[:,:]]=1
print predict

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  0.  0.]]


In [72]:
print (3.*predict.shape[0]-np.sum(predict))/predict.shape[0]

0.491786958686


In [ ]:
print np.sum(predict)/np.sum(testLabels[:,0])

In [79]:
testImages, testLabels = init.loadData(cfg)
testImages, testLabels = cleanData(testImages, testLabels)

LOADING DATA...
NNN
N
SSS
LLLL
N
N
L
NN
SSSS
SS
N
SSSS
N
NN
NN
NN
N
NN
NNNN
NS
N
NN
LL
N
N
N
NNNN
LLL
NS
N
NNNN
N
NS
LLLL
LLSS
NN
LLN
NNN
NN
LLLL
SSSS
N
SSSS
N
SSS
NN
SS
NN
NN
NNN
LSS
N
S
NNNN
S
NN
NN
NN
S
N
L
SSS
L
S
NNN
S
N
NSS
N
SSS
NN
S
N
NN
NSSS
SSS
L
SSSS
NN
L
S
S
S
N
NNNN
LNSS
N
N
N
L
N
SS
SS
SS
SSSS
SS
SSSS
L
LSSS
SS
SS
SSS
NN
SSS
NN
L
NSS
SSS
NN
N
NN
NSSS
SSS
N
SS
SSSS
S
NN
N
SSS
SSS
SSS
NNN
NN
N
NN
N
LLL
SSS
NN
N
SSSS
L
S
L
N
NN
NNSS
S
NN
N
LS
NN
LLS
LLS
SSSS
N
NN
NNN
NN
LSS
NN
NNN
NNN
NNN
S
NN
NN
SS
SSSS
LL
NSSS
S
N
N
LLL
NNN
NN
LSSS
NN
NSSS
SSSS
NN
SSS
NNNN
SS
NNN
NN
N
LLN
NSS
SSSS
NN
NNN
N
N
NSS
SS
S
NN
NN
LLL
S
S
N
N
NN
NNNN
LLLS
LLS
SSS
NNS
LLLS
L
LL
N
NNS
SS
NN
LL
NNNN
L
SSSS
NN
SSSS
NN
L
NN
N
SSSS
N
NSS
SS
N
NS
SSS
LLLL
NN
LS
LS
LLS
SSSS
NN
NN
SSS
LL
SSSS
NNNN
NN
NNNN
N
SSS
NN
SSS
S
N
NN
N
N
NNS
LLS
NNN
SSSS
N
NN
SSSS
SS
SSS
LLLS
SSS
NN
S
NNNN
N
SSS
NNN
SSS
SS
NNN
NNNS
SSS
N
NNN
SS
SS
NN
NN
S
NNNN
NN
NNN
NNNS
SSS
NNS
NN
NNN
SS
N
SSSS
NN
NN
SSSS
NNN
N
NN
NN
NN
LLLL
NN
L

In [80]:
testImages = ku.to3d(testImages)
testLabels = get3Labels(testLabels)
testImages = ku.convertArray(testImages)

NNN
SSS
LLLL
NN
SSSS
SS
SSSS
NN
NN
NN
NN
NNNN
NN
LL
NNNN
LLL
NNNN
LLLL
NN
NNN
NN
LLLL
SSSS
SSSS
SSS
NN
SS
NN
NN
NNN
NNNN
NN
NN
NN
SSS
NNN
SSS
NN
NN
SSS
SSSS
NN
NNNN
SS
SS
SS
SSSS
SS
SSSS
SS
SS
SSS
NN
SSS
NN
SSS
NN
NN
SSS
SS
SSSS
NN
SSS
SSS
SSS
NNN
NN
NN
LLL
SSS
NN
SSSS
NN
NN
NN
SSSS
NN
NNN
NN
NN
NNN
NNN
NNN
NN
NN
SS
SSSS
LL
LLL
NNN
NN
NN
SSSS
NN
SSS
NNNN
SS
NNN
NN
SSSS
NN
NNN
SS
NN
NN
LLL
NN
NNNN
SSS
LL
SS
NN
LL
NNNN
SSSS
NN
SSSS
NN
NN
SSSS
SS
SSS
LLLL
NN
SSSS
NN
NN
SSS
LL
SSSS
NNNN
NN
NNNN
SSS
NN
SSS
NN
NNN
SSSS
NN
SSSS
SS
SSS
SSS
NN
NNNN
SSS
NNN
SSS
SS
NNN
SSS
NNN
SS
SS
NN
NN
NNNN
NN
NNN
SSS
NN
NNN
SS
SSSS
NN
NN
SSSS
NNN
NN
NN
NN
LLLL
NN
LLLL
SSSS
NNN
NN
NN
NNNN
SSSS
SSSS
SS
NN
NN
NN
SSS
SSS
SSS
LLLL
NNNN
SSS
NNNN
NNN
SSS
NN
NNN
NNN
LLLL
NNN
SS
SS
SSS
SS
NN
SS
NN
NNN
SS
LL
NN
NNN
NNN
LLLL
SS
NN
NNN
NNNN
SSS
LL
SSS
LL
NN
NN
NNN
NNNN
NN
LL
NN
NN
NN
NN
SSSS
LL
SSSS
SSSS
NN
NN
LL
SS
NNN
NNN
SSSS
SSSS
SSS
LLLL
SS
SSS
NNN
NNN
NN
SSS
LLLL
SSS
NNN
SSSS
SSSS
NN
NN
NN
NN
NN
NN
NN
SS
NNN
SSS
NN